# Capstone Project
The present notebook documents the start-to-finish Data Science Process—from preprocessing and exploration to modeling and interpreting—as it pertains to this **Capstone Project**, done in culmination of the Part-Time Online Data Science Course with Flatiron School.


* Student name: Tim Sennett
* Student pace: self paced / **part time** / full time
* Scheduled project review date/time: ... AM EDT
* Instructor name: Jeff Herman
* Blog post URL: https://medium.com/@timsennett/

## Proposal

Provide thesis... Ultimately, this work aims to predict when a pitcher (Chris Sale)...

A full list of features and their respective descriptions have been placed in the `columns.txt` file of this repository.

but by using underlying pitch metrics (e.g., `release_speed`, `release_spin_rate`) as a proxy, 

In [ ]:
# include link to YouTube video on baseball savant search demos

In [ ]:
# explain what statcast is

In [ ]:
# don't include result of pitch or plate appearance, because it's too noisy.
# could be the result of poor defense, bad calls, random chance

### Following the [OSEMN framework](https://towardsdatascience.com/5-steps-of-a-data-science-project-lifecycle-26c50372b492), the present notebook is organized according to the following basic steps:

1. Obtain
2. Scrub
3. Explore
4. Model
5. Interpret

## 1. Obtain
#### Gathering data from the relevant sources.

The data for this project was obtained from [Baseball Savant](https://baseballsavant.mlb.com/statcast_search). It includes a zip drive containing two folders: train and test samples of chest X-ray images.

There are 5232 files in the train folder, and 624 images in the test folder. The samples are labeled as either `Normal` or `Pneumonia`. I manually created a third folder to store a validation set, which I obtained by manually moving 32 files from the test folder (16 from each of the two classes, including an equal number of viral and bacterial pneumonia samples).

In [1046]:
# Import libraries that I anticipate using

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In the next cell, I'll check to see what's in the current repository. I want to be sure to load the appropriate data.

In [1047]:
ls

README.md                           online_capstone_project_rubric.pdf
columns.txt                         savant_data_sale.csv
future_work.txt                     savant_data_verlander.csv
img/                                student.ipynb


I'm going to begin with the `savant_data_sale.csv` file, so I'll now import that as a Pandas DataFrame.

In [1048]:
sale = pd.read_csv('savant_data_sale.csv')

In [1049]:
# Check to see that my data imported properly
sale.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,CH,2019-08-13,89.8,3.1853,4.9191,Chris Sale,596019,519242,double,hit_into_play_score,NaN,NaN,NaN,NaN,8.0,Francisco Lindor doubles (28) on a line drive ...,R,R,L,CLE,BOS,X,7.0,line_drive,0,1,2019,1.4110,-0.0924,-0.0889,1.4573,547379.0,656185.0,NaN,2,7,Bot,55.97,110.57,NaN,NaN,506702.0,NaN,190814_014412,-10.9405,-130.2111,-2.3558,18.0909,26.2440,-32.9637,3.2900,1.5500,118.0,107.8,5.0,89.118,1983.0,5.973,565379,519242,506702.0,519048.0,593523.0,646240.0,593428.0,643217.0,598265.0,605141.0,54.5262,0.642,0.585,1.25,1.0,1.0,1.0,4.0,64,2,Changeup,3,6,3,6,6,3,3,6,Standard,Standard
1,SL,2019-08-13,81.3,3.2220,5.2825,Chris Sale,596019,519242,NaN,called_strike,NaN,NaN,NaN,NaN,12.0,NaN,R,R,L,CLE,BOS,S,NaN,NaN,0,0,2019,-1.0911,-0.3883,0.8851,3.0959,547379.0,656185.0,NaN,2,7,Bot,NaN,NaN,NaN,NaN,506702.0,NaN,190814_014344,-3.1605,-118.1095,2.4188,-9.2399,23.3693,-36.4355,3.4565,1.5855,NaN,NaN,NaN,79.805,2624.0,5.435,565379,519242,506702.0,519048.0,593523.0,646240.0,593428.0,643217.0,598265.0,605141.0,55.0647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64,1,Slider,3,6,3,6,6,3,3,6,Standard,Standard
2,SL,2019-08-13,83.1,3.0410,5.2492,Chris Sale,571980,519242,field_out,hit_into_play,NaN,NaN,NaN,NaN,5.0,Tyler Naquin pops out to first baseman Mitch M...,R,L,L,CLE,BOS,X,3.0,popup,2,2,2019,-1.1546,-0.4757,-0.1503,2.2940,547379.0,656185.0,NaN,1,7,Bot,167.72,156.28,NaN,NaN,506702.0,NaN,190814_014256,-5.0542,-120.9725,0.6173,-10.2351,23.8790,-37.1534,3.4100,1.6200,146.0,72.5,62.5,82.835,2636.0,6.117,565379,519242,506702.0,519048.0,593523.0,646240.0,593428.0,643217.0,598265.0,605141.0,54.3824,0.015,0.017,0.00,1.0,0.0,0.0,3.0,63,5,Slider,3,6,3,6,6,3,3,6,Standard,Standard
3,FF,2019-08-13,96.6,3.0312,5.5203,Chris Sale,571980,519242,NaN,swinging_strike,NaN,NaN,NaN,NaN,2.0,NaN,R,L,L,CLE,BOS,S,NaN,NaN,2,1,2019,0.9229,0.9892,-0.0025,3.1884,547379.0,656185.0,NaN,1,7,Bot,NaN,NaN,NaN,NaN,506702.0,NaN,190814_014222,-10.2223,-140.3095,-2.9091,14.5585,31.2505,-18.5637,3.4100,1.6200,NaN,NaN,NaN,96.925,2422.0,6.540,565379,519242,506702.0,519048.0,593523.0,646240.0,593428.0,643217.0,598265.0,605141.0,53.9592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63,4,4-Seam Fastball,3,6,3,6,6,3,3,6,Standard,Standard
4,FF,2019-08-13,95.7,3.1314,5.2087,Chris Sale,571980,519242,NaN,called_strike,NaN,NaN,NaN,NaN,1.0,NaN,R,L,L,CLE,BOS,S,NaN,NaN,2,0,2019,1.1255,0.9681,-0.5661,2.9124,547379.0,656185.0,NaN,1,7,Bot,NaN,NaN,NaN,NaN,506702.0,NaN,190814_014153,-12.3400,-138.8856,-2.6033,17.4733,31.9428,-19.1956,3.3321,1.5286,NaN,NaN,NaN,95.798,2525.0,6.554,565379,519242,506702.0,519048.0,593523.0,646240.0,593428.0,643217.0,598265.0,605141.0,53.9453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63,3,4-Seam Fastball,3,6,3,6,6,3,3,6,Standard,Standard


## 2. Scrub / Preprocessing
#### Clean data to a format suitable for machine learning and time-series analysis.

In the next few cells, I'll look to understand the dimensionality of my data, and get a feel for what sort of data it contains. I'll also be on the lookout for signs of any missing data.

In [1050]:
sale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15173 entries, 0 to 15172
Data columns (total 89 columns):
pitch_type                         15094 non-null object
game_date                          15173 non-null object
release_speed                      15163 non-null float64
release_pos_x                      15036 non-null float64
release_pos_z                      15036 non-null float64
player_name                        15173 non-null object
batter                             15173 non-null int64
pitcher                            15173 non-null int64
events                             3854 non-null object
description                        15173 non-null object
spin_dir                           0 non-null float64
spin_rate_deprecated               0 non-null float64
break_angle_deprecated             0 non-null float64
break_length_deprecated            0 non-null float64
zone                               15163 non-null float64
des                                3854 non-nul

The DataFrame includes 89 total features and 15,173 observations. A detailed understanding of these columns can be gleaned from the [Statcast Search CSV Documentation](https://baseballsavant.mlb.com/csv-docs), which includes a description for each feature.

Right away, I know that several of these features are going to be of no help in answering my question, because they're irrelevant for some reason or another. In many cases, these features contain information about the on-field result of a given pitch rather than the underlying pitch information that I'm interested in using. I'll go ahead and remove those, keeping only the features that potentially have some value in targeting pitcher fatigue in a manner independent from on-field results:

- `player_name`
- `batter`
- `pitcher`
- `events`
- `description`
- `spin_dir`
- `spin_rate_deprecated`
- `break_angle_deprecated`
- `break_length_deprecated`
- `des`
- `game_type`
- `stand`
- `p_throws`
- `home_team`
- `away_team`
- `type`
- `hit_location`
- `balls`
- `strikes`
- `game_year`
- `on_3b`
- `on_2b`
- `on_1b`
- `outs_when_up`
- `inning_topbot`
- `tfs_deprecated`
- `tfs_zulu_deprecated`
- `fielder_2`
- `umpire`
- `sv_id`
- `sz_top`
- `sz_bot`
- `game_pk`
- `pitcher.1`
- `fielder_2.1`
- `fielder_3`
- `fielder_4`
- `fielder_5`
- `fielder_6`
- `fielder_7`
- `fielder_8`
- `fielder_9`
- `woba_value`
- `woba_denom`
- `babip_value`
- `iso_value`
- `home_score`
- `away_score`
- `bat_score`
- `fld_score`
- `post_home_score`
- `post_away_score`
- `post_bat_score`
= `post_fld_score`
- `if_fielding_alignment`
- `of_fielding_alignment`

In [1051]:
sale.drop(columns=['player_name', 'batter', 'pitcher', 'events', 'spin_dir',
                   'spin_rate_deprecated', 'break_angle_deprecated',
                   'break_length_deprecated', 'des', 'game_type', 'p_throws',
                   'home_team', 'away_team', 'hit_location', 'balls',
                   'strikes', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
                   'inning_topbot', 'tfs_deprecated', 'tfs_zulu_deprecated',
                   'fielder_2', 'umpire', 'sv_id', 'sz_top', 'sz_bot',
                   'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3',
                   'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7',
                   'fielder_8', 'fielder_9', 'woba_value', 'woba_denom',
                   'babip_value', 'iso_value', 'home_score', 'away_score',
                   'bat_score', 'fld_score', 'post_home_score',
                   'post_away_score', 'post_bat_score', 'post_fld_score',
                   'if_fielding_alignment', 'of_fielding_alignment',
                   'stand', 'type', 'game_year', 'description'],
          inplace=True)

In [1052]:
# preview updated data

sale.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,zone,bb_type,pfx_x,pfx_z,plate_x,plate_z,inning,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,launch_speed_angle,at_bat_number,pitch_number,pitch_name
0,CH,2019-08-13,89.8,3.1853,4.9191,8.0,line_drive,1.4110,-0.0924,-0.0889,1.4573,7,55.97,110.57,-10.9405,-130.2111,-2.3558,18.0909,26.2440,-32.9637,118.0,107.8,5.0,89.118,1983.0,5.973,54.5262,0.642,0.585,4.0,64,2,Changeup
1,SL,2019-08-13,81.3,3.2220,5.2825,12.0,NaN,-1.0911,-0.3883,0.8851,3.0959,7,NaN,NaN,-3.1605,-118.1095,2.4188,-9.2399,23.3693,-36.4355,NaN,NaN,NaN,79.805,2624.0,5.435,55.0647,NaN,NaN,NaN,64,1,Slider
2,SL,2019-08-13,83.1,3.0410,5.2492,5.0,popup,-1.1546,-0.4757,-0.1503,2.2940,7,167.72,156.28,-5.0542,-120.9725,0.6173,-10.2351,23.8790,-37.1534,146.0,72.5,62.5,82.835,2636.0,6.117,54.3824,0.015,0.017,3.0,63,5,Slider
3,FF,2019-08-13,96.6,3.0312,5.5203,2.0,NaN,0.9229,0.9892,-0.0025,3.1884,7,NaN,NaN,-10.2223,-140.3095,-2.9091,14.5585,31.2505,-18.5637,NaN,NaN,NaN,96.925,2422.0,6.540,53.9592,NaN,NaN,NaN,63,4,4-Seam Fastball
4,FF,2019-08-13,95.7,3.1314,5.2087,1.0,NaN,1.1255,0.9681,-0.5661,2.9124,7,NaN,NaN,-12.3400,-138.8856,-2.6033,17.4733,31.9428,-19.1956,NaN,NaN,NaN,95.798,2525.0,6.554,53.9453,NaN,NaN,NaN,63,3,4-Seam Fastball


In [1053]:
# see column and null value counts

print("There are {} columns remaining.".format(len(sale.columns)))
sale.isnull().sum()

There are 33 columns remaining.


pitch_type                            79
game_date                              0
release_speed                         10
release_pos_x                        137
release_pos_z                        137
zone                                  10
bb_type                            12868
pfx_x                                 10
pfx_z                                 10
plate_x                               10
plate_z                               10
inning                                 0
hc_x                               12868
hc_y                               12868
vx0                                   10
vy0                                   10
vz0                                   10
ax                                    10
ay                                    10
az                                    10
hit_distance_sc                    11687
launch_speed                       11432
launch_angle                       11432
effective_speed                      172
release_spin_rat

That leaves me with 33 columns.

Let me now take some further inventory of these remaining features, and see there's anything else worth removing.

It appears there are several columns where null values abound. Particularly, I'm referring to...

- `bb_type`
- `hc_x`
- `hc_y`
- `hit_distance_sc`
- `launch_speed`
- `launch_angle`
- `estimated_ba_using_speedangle`
- `estimated_woba_using_speedangle`
- `launch_speed_angle`

Unsurprisingly, these are features that are dependent on the given observation, or pitch, being hit by the batter into the field of play. Since most pitches are not hit into play (Indeed, this can technically only occur with the final pitch of a Plate Appearance [`at_bat_number`]), and because the aim of this project is to target pitcher fatigue in a manner independent from on-field results, I'm going to want to remove these columns as well.

Upon even further inspection, I notice that there are a handful of other features that are mostly unrelated to pitcher fatigue, and that, in some cases, are likely to be heavily influenced by the individual batter to whom a  pitch is thrown. For example, `plate_x`, which offers the horizontal position of the ball when it crosses home plate (from the catcher's perspective), may have less to say about a pitcher's stamina/fatigue than it does about such things as a game-plan, pitch-sequencing, a scouting report, the handedness of the batter, etc. With that said, it may be wise to remove these columns as well:

- `zone`
- `plate_x`
- `plate_z`

In [1054]:
sale.drop(columns=['bb_type', 'hc_x', 'hc_y', 'hit_distance_sc',
                   'launch_speed', 'launch_angle',
                   'estimated_ba_using_speedangle',
                   'estimated_woba_using_speedangle', 'launch_speed_angle',
                   'zone', 'plate_x', 'plate_z'],
          inplace=True)

In [1055]:
print("There are {} columns remaining.".format(len(sale.columns)))
sale.isnull().sum()

There are 21 columns remaining.


pitch_type            79
game_date              0
release_speed         10
release_pos_x        137
release_pos_z        137
pfx_x                 10
pfx_z                 10
inning                 0
vx0                   10
vy0                   10
vz0                   10
ax                    10
ay                    10
az                    10
effective_speed      172
release_spin_rate    222
release_extension    170
release_pos_y        137
at_bat_number          0
pitch_number           0
pitch_name            79
dtype: int64

That leaves me with 21 columns. At this point, I've removed all of the ones that were majority-NaN. Now it's time to consider what I should do with the missing values in the rest of the columns where at least some remain present. A few observations:

- Judging from their respective descriptions found in the [documentation](https://baseballsavant.mlb.com/csv-docs), the features `pitch_type` and `pitch_name` may offer completely redundant information.
- `release_pos_x`, `release_pos_y`, and `release_pos_z` each contain 137 missing values. I assume these are all tied to the same 137 pitches.
- There are twelve columns that contain exactly ten missing values. I imagine this is occurring for the same ten pitches, but I'm curious to know for sure.

First, I'll quickly look into `pitch_type` and `pitch_name`.

In [1056]:
sale[['pitch_type', 'pitch_name']].head(10)

,pitch_type,pitch_name
0,CH,Changeup
1,SL,Slider
2,SL,Slider
3,FF,4-Seam Fastball
4,FF,4-Seam Fastball
5,SL,Slider
6,SL,Slider
7,CH,Changeup
8,SL,Slider
9,CH,Changeup


In [1057]:
# STOP:

# This would work better as a visualization
# Or, just add a visualization alongside it

In [1058]:
print("There are {} unique pitch types.".format(sale['pitch_type'].nunique()))
print("There are {} unique pitch names.".format(sale['pitch_name'].nunique()),
     '\n')
print('Unique pitch types:')
print(sale['pitch_type'].unique(),'\n')
print('Unique pitch names:')
print(sale['pitch_name'].unique())

There are 8 unique pitch types.
There are 8 unique pitch names. 

Unique pitch types:
['CH' 'SL' 'FF' 'FT' nan 'UN' 'IN' 'FA' 'FS'] 

Unique pitch names:
['Changeup' 'Slider' '4-Seam Fastball' '2-Seam Fastball' nan 'Unknown'
 'Intentional Ball' 'Fastball' 'Split Finger']


In [1059]:
# Compare value-counts in 'pitch_type' and 'pitch_name' columns

print(sale.pitch_type.value_counts(),'\n')
print(sale.pitch_name.value_counts())

FT    4475
SL    4445
FF    3135
CH    3033
FA       2
UN       2
IN       1
FS       1
Name: pitch_type, dtype: int64 

2-Seam Fastball     4475
Slider              4445
4-Seam Fastball     3135
Changeup            3033
Unknown                2
Fastball               2
Intentional Ball       1
Split Finger           1
Name: pitch_name, dtype: int64


Evidently, these columns contain virtually identical information, where `CH` in `pitch_type` corresponds to `Changeup` in `pitch_name`, and so on.

I'll remove `pitch_type`, because I prefer the unabbreviated values in `pitch_name`.

In [1060]:
sale.drop(columns=['pitch_type'], inplace=True)

While I'm here, I'll also take this opportunity to review the lone "intentional ball" in the data, categorized under `pitch_name` as `Intentional Ball`. Traditionally, four "intentional balls"—when thrown in a single Plate Appearance—comprise one "intentional walk". It would be fairly usually to see only one, stand-alone intentional ball, as appears might be the case here.

Some potentially important context:

For decades, intentional walks have been issued as a matter of strategy in select situations when the fielding team considers it to be advantageous that the current batter pass freely to first base. These pitches are never meant to be competitive, as they are almost always thrown beyond the batter's reach and with very low velocity.

Up until 2017, intentional balls were required to be thrown (more likely lobbed) to the catcher, who would be standing wide of home plate, as seen in the  below photograph.

In [1061]:
# STOP

# can I resize this
# do I need to credit it?

![](img/intentionalball.jpg)

However, prior to the beginning of the 2017 season, the official rules were amended to allow a manager to order an intentional walk by simply signaling to the umpire from the dugout, eliminating the need for the pitcher to actually throw any intentional balls.

With that said, I'm going to take a look at the intentional ball that Christ Sale threw. I'm curious to know when it was thrown. That way, I can then see if it indeed was a standalone incident, or if it's situated around other intentional balls that might have been miscategorized.

In [1062]:
# find any pitch in the data labeled as an intentional ball

sale[sale['pitch_name']=='Intentional Ball']

,game_date,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,inning,vx0,vy0,vz0,ax,ay,az,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name
9731,2016-07-28,47.6,NaN,NaN,0.839992,1.488567,6,2.515,-69.128,4.902,3.338,8.474,-27.795,NaN,NaN,NaN,NaN,48,2,Intentional Ball


Now, let me see what transpired during that particular Plate Appearance (see `at_bat_number`) on July 28, 2016.

In [1063]:
# make subset of relevant date
july_28_2016 = sale[sale['game_date']=='2016-07-28']

# display all pitches from relevant plate appearance
july_28_2016[july_28_2016['at_bat_number']==48]

,game_date,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,inning,vx0,vy0,vz0,ax,ay,az,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name
9729,2016-07-28,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,4,NaN
9730,2016-07-28,43.8,NaN,NaN,0.391875,2.016033,6,1.174,-63.664,4.148,1.531,6.947,-26.934,NaN,NaN,NaN,NaN,48,3,Unknown
9731,2016-07-28,47.6,NaN,NaN,0.839992,1.488567,6,2.515,-69.128,4.902,3.338,8.474,-27.795,NaN,NaN,NaN,NaN,48,2,Intentional Ball
9732,2016-07-28,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,1,NaN


It indeed looks as if four intentional balls were thrown, but only one was categorized as such. (Two show up as missing, and one is labeled as `Unknown`.)

Because there's no reason to think that the pitch metrics associated with an intentional ball will be helpful in answering my question, I'll remove this particular row where `Intentional Ball` is under `pitch_name`.

While I'm at it, I'll also drop the small handful rows (five total) in the dataset where the `pitch_name` is labeled as `Unknown`, `Fastball`, or `Split Finger`, because I'm now sure that `Unknown` is too much of a wild-card to be of any use, and because the data makes clear that Chris Sale is, for all intents and purposes, a four-pitch pitcher:

In [1064]:
# STOP make visualize of pitch distribution

In [1065]:
# STOP is there a shorter way to write this?
# drop rows of lesser-used pitch_names

sale = sale[sale['pitch_name']!='Intentional Ball']
sale = sale[sale['pitch_name']!='Unknown']
sale = sale[sale['pitch_name']!='Fastball']
sale = sale[sale['pitch_name']!='Split Finger']

In [1066]:
# sanity check
sale.pitch_name.value_counts()

2-Seam Fastball    4475
Slider             4445
4-Seam Fastball    3135
Changeup           3033
Name: pitch_name, dtype: int64

Okay, no more intentional balls; now the `pitch_name` values are reduced to the four pitches that Sale almost exclusively throws.

After removing those rows, let me get an updated look on my null-value counts.

In [1067]:
sale.isnull().sum()

game_date              0
release_speed         10
release_pos_x        134
release_pos_z        134
pfx_x                 10
pfx_z                 10
inning                 0
vx0                   10
vy0                   10
vz0                   10
ax                    10
ay                    10
az                    10
effective_speed      169
release_spin_rate    219
release_extension    167
release_pos_y        134
at_bat_number          0
pitch_number           0
pitch_name            79
dtype: int64

Now, I'll look to see if the 134 missing values in `release_pos_x` are associated with the 134 missing values in `release_pos_y` and `release_pos_z`, as I suspect they are.

In [1068]:
# view subset of df only where `release_pos_x` is NaN

# remove limit on display rows
pd.set_option('display.max_rows', None)

sale[sale['release_pos_x'].isnull()]

,game_date,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,inning,vx0,vy0,vz0,ax,ay,az,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name
2235,2019-04-09,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,3,NaN
4467,2018-04-26,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,4,NaN
5658,2017-08-24,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,4,NaN
5823,2017-08-13,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44,3,NaN
8181,2017-04-15,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,3,Slider
8521,2016-09-27,95.2,NaN,NaN,1.353517,0.875100,6,-9.568,-138.290,-1.892,20.469,33.220,-23.063,NaN,NaN,NaN,NaN,56,4,2-Seam Fastball
8530,2016-09-27,76.6,NaN,NaN,-0.953867,-0.548200,6,-6.517,-111.377,0.367,-7.346,22.123,-38.586,NaN,NaN,NaN,NaN,54,3,Slider
8611,2016-09-27,89.4,NaN,NaN,1.320117,0.602767,1,-12.125,-129.698,-1.047,17.501,30.782,-27.377,NaN,NaN,NaN,NaN,1,1,Changeup
8729,2016-09-16,84.9,NaN,NaN,1.755708,0.710267,6,-14.068,-122.827,-1.927,20.341,28.295,-26.756,NaN,NaN,NaN,NaN,49,1,Changeup
8731,2016-09-16,77.2,NaN,NaN,-0.701975,-0.373333,6,-4.820,-112.383,2.582,-5.140,24.397,-37.111,NaN,NaN,NaN,NaN,47,1,Slider


In [1069]:
# put back limit on display rows
pd.set_option('display.max_rows', 60)

Clearly, wherever `release_pos_x` shows a missing value there's also a missing value under `release_pos_y` and `release_pos_z` (and in several other columns as well, it appears).

Because these pitches represent a very small portion of the data overall, and because so many of them present null values in multiple columns, one viable solution could be to drop these 134 rows.

Instead, in the interest of holding onto as much data as I responsibly can, I think I would do well to replace most of these null values using the `ffill` method, which propagates the last valid observation forward to the next valid observation. There's a certain intuition to this. For example, the best indicator of what will be the velocity of the next fastball is probably the velocity of the previous fastball, and so on.

Obviously, this only make sense if it's done with pitch-type in mind. (If consecutive pitches are of different types, the metrics of one will likely have little in common with the metrics of another.)

In [1070]:
# divide data into four subsets per pitch_name
changeups = sale[sale['pitch_name']=='Changeup']
sliders = sale[sale['pitch_name']=='Slider']
four_seam_ff = sale[sale['pitch_name']=='4-Seam Fastball']
two_seam_ff = sale[sale['pitch_name']=='2-Seam Fastball']

# fifth subset for null pitch_name
pitch_type_na = sale[sale['pitch_name'].isnull()]

In [1071]:
# fill null values with ffill method
changeups.fillna(method='ffill', inplace=True)
sliders.fillna(method='ffill', inplace=True)
four_seam_ff.fillna(method='ffill', inplace=True)
two_seam_ff.fillna(method='ffill', inplace=True)

In [1124]:
# concatenate five subsets
sale = pd.concat([changeups, sliders, four_seam_ff, two_seam_ff,
                  pitch_type_na])

# sort index
sale.sort_index(inplace=True)

sale.head()

,game_date,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,inning,vx0,vy0,vz0,ax,ay,az,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name
0,2019-08-13,89.8,3.1853,4.9191,1.4110,-0.0924,7,-10.9405,-130.2111,-2.3558,18.0909,26.2440,-32.9637,89.118,1983.0,5.973,54.5262,64,2,Changeup
1,2019-08-13,81.3,3.2220,5.2825,-1.0911,-0.3883,7,-3.1605,-118.1095,2.4188,-9.2399,23.3693,-36.4355,79.805,2624.0,5.435,55.0647,64,1,Slider
2,2019-08-13,83.1,3.0410,5.2492,-1.1546,-0.4757,7,-5.0542,-120.9725,0.6173,-10.2351,23.8790,-37.1534,82.835,2636.0,6.117,54.3824,63,5,Slider
3,2019-08-13,96.6,3.0312,5.5203,0.9229,0.9892,7,-10.2223,-140.3095,-2.9091,14.5585,31.2505,-18.5637,96.925,2422.0,6.540,53.9592,63,4,4-Seam Fastball
4,2019-08-13,95.7,3.1314,5.2087,1.1255,0.9681,7,-12.3400,-138.8856,-2.6033,17.4733,31.9428,-19.1956,95.798,2525.0,6.554,53.9453,63,3,4-Seam Fastball


In [1125]:
# recheck null value totals
sale.isnull().sum()

game_date             0
release_speed         9
release_pos_x         9
release_pos_z         9
pfx_x                 9
pfx_z                 9
inning                0
vx0                   9
vy0                   9
vz0                   9
ax                    9
ay                    9
az                    9
effective_speed       9
release_spin_rate    10
release_extension     9
release_pos_y         9
at_bat_number         0
pitch_number          0
pitch_name           79
dtype: int64

With that step now complete, I now see just a small number of missing values in most columns. My hunch in that these null values are tied to an uncategorized (`NaN`) pitch-type. Let me check more closely.

Calling for the observations where null values are present under `release_spin_rate` should show me everything I need to know.

In [1126]:
sale[sale['release_spin_rate'].isnull()]

,game_date,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,inning,vx0,vy0,vz0,ax,ay,az,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name
2235,2019-04-09,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,3,NaN
4467,2018-04-26,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,4,NaN
5658,2017-08-24,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,4,NaN
5823,2017-08-13,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44,3,NaN
9729,2016-07-28,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,4,NaN
9732,2016-07-28,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,1,NaN
10402,2016-06-15,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,4,NaN
10403,2016-06-15,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,3,NaN
10404,2016-06-15,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,2,NaN
10604,2016-06-04,75.3,3.0571,5.158,-1.079117,-0.6127,5,-4.105,-109.587,0.425,-8.304,19.136,-39.006,74.115,NaN,5.547,54.9543,39,3,NaN


Most of these rows are utterly unsalvagable because virtually every meaningful feature shows a missing value. I'll have no choice other than to remove them.

There is one row where all that's missing is `release_spin_rate`. The problem, of course, is that no `pitch_name` is given, so even something as simple as plugging in a median value makes little sense.

On the surface, I notice that the `release_speed` for this pitch is 75.3 mph, which is certainly on the slow side, especially for Chris Sale. Just to make sure, I'll inspect the average speeds for each of Chris Sale's pitch-types.

In [1127]:
# STOP include graph showing release_speed per pitch name

pitch_names = ['Changeup', 'Slider', '4-Seam Fastball', '2-Seam Fastball']

for name in pitch_names:
    print(f"Median release speed for {name}: {sale[sale['pitch_name']==name]['release_speed'].median()}")

Median release speed for Changeup: 86.5
Median release speed for Slider: 79.6
Median release speed for 4-Seam Fastball: 94.7
Median release speed for 2-Seam Fastball: 94.2


If anything, this pitch in question seems likely to be a slider. Sliders, by the way, are known for having a sweeping, horizontal movement as it reaches home plate. The telltale sign would be if this pitch has horizontal movement (`pfx_x`) that lines up well with Sale's other sliders. Do any of Sale's pitch-types have an average `pfx_x` close to -1.079117?

In [1128]:
# # STOP include graph showing pfx_x per pitch name

for name in pitch_names:
    print(f"Median horizontal movement for {name}: {sale[sale['pitch_name']==name]['pfx_x'].median()}")

Median horizontal movement for Changeup: 1.58175
Median horizontal movement for Slider: -1.0649
Median horizontal movement for 4-Seam Fastball: 1.2251
Median horizontal movement for 2-Seam Fastball: 1.460675


No doubt, this pitch is a slider! Let's give it a name, then forward-fill a `release_spin_rate`.

In [1129]:
# replace null value with the string 'Slider'
sale.at[10604, 'pitch_name'] = 'Slider'

In [1130]:
# sanity check
sale.loc[10604]

game_date            2016-06-04
release_speed              75.3
release_pos_x            3.0571
release_pos_z             5.158
pfx_x                  -1.07912
pfx_z                   -0.6127
inning                        5
vx0                      -4.105
vy0                    -109.587
vz0                       0.425
ax                       -8.304
ay                       19.136
az                      -39.006
effective_speed          74.115
release_spin_rate           NaN
release_extension         5.547
release_pos_y           54.9543
at_bat_number                39
pitch_number                  3
pitch_name               Slider
Name: 10604, dtype: object

In [1132]:
# subset for sliders
sliders = sale[sale['pitch_name']=='Slider']

# fill lone missing 'release_spin_rate' value
sliders.fillna(method='ffill', inplace=True)

# update df to reflect change
sale.update(sliders)

In [1133]:
# recheck null totals
sale.isnull().sum()

game_date             0
release_speed         9
release_pos_x         9
release_pos_z         9
pfx_x                 9
pfx_z                 9
inning                0
vx0                   9
vy0                   9
vz0                   9
ax                    9
ay                    9
az                    9
effective_speed       9
release_spin_rate     9
release_extension     9
release_pos_y         9
at_bat_number         0
pitch_number          0
pitch_name           78
dtype: int64

Almost there. The next step is to drop the nine rows that are mostly filled with missing data.

In [1134]:
sale.dropna(subset=['release_speed'], inplace=True)

In [1135]:
# recheck null totals
sale.isnull().sum()

game_date             0
release_speed         0
release_pos_x         0
release_pos_z         0
pfx_x                 0
pfx_z                 0
inning                0
vx0                   0
vy0                   0
vz0                   0
ax                    0
ay                    0
az                    0
effective_speed       0
release_spin_rate     0
release_extension     0
release_pos_y         0
at_bat_number         0
pitch_number          0
pitch_name           69
dtype: int64

After all of that, I'm left with just 69 missing pitch names. Let me see when those show up.

In [1138]:
# see dates when 'pitch_name' is NaN
sale[sale['pitch_name'].isnull()].game_date.value_counts()

2016-06-04    66
2015-09-07     3
Name: game_date, dtype: int64

All but three of these 69 null values occurred on June 4, 2016. Something must've gone wrong with Statcast that day! I'm interested in viewing a subset of the data that includes only that day's pitches.

In [1140]:
# remove limit on display rows
pd.set_option('display.max_rows', None)

sale[sale['game_date']=='2016-06-04']

,game_date,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,inning,vx0,vy0,vz0,ax,ay,az,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name
10569,2016-06-04,95.3,3.0566,5.3567,1.090492,1.166067,7.0,-9.472,-138.456,-2.470,17.181,28.400,-18.895,94.613,2328.0,6.003,54.4991,58.0,7.0,NaN
10570,2016-06-04,81.8,2.9592,5.3189,-1.134783,-0.578300,7.0,-8.446,-118.751,0.600,-10.164,25.713,-39.748,79.536,2269.0,5.332,55.1696,58.0,6.0,NaN
10571,2016-06-04,95.6,2.9721,5.6498,1.322900,1.154600,7.0,-10.556,-138.820,-1.674,20.208,33.506,-19.203,94.317,2402.0,5.975,54.5269,58.0,5.0,NaN
10572,2016-06-04,95.2,2.8542,5.2797,1.190692,1.117333,7.0,-8.706,-138.248,-1.388,18.358,31.065,-19.731,94.716,2369.0,6.311,54.1903,58.0,4.0,NaN
10573,2016-06-04,92.4,3.1507,5.6105,1.279758,0.658667,7.0,-6.764,-134.474,-1.859,18.444,30.781,-26.273,90.965,2150.0,5.688,54.8138,58.0,3.0,NaN
10574,2016-06-04,94.4,2.9587,5.4278,1.448150,1.075767,7.0,-13.303,-136.716,-5.533,21.697,27.000,-20.414,93.408,2309.0,5.907,54.5952,58.0,2.0,NaN
10575,2016-06-04,94.8,2.9834,5.2266,1.136417,1.279300,7.0,-12.584,-137.469,-2.881,17.733,25.191,-17.426,94.992,2322.0,6.421,54.0804,58.0,1.0,NaN
10576,2016-06-04,88.9,3.1527,5.2244,1.040392,0.697367,7.0,-9.755,-129.063,-2.178,14.347,24.061,-26.190,88.438,1883.0,6.071,54.4309,57.0,3.0,NaN
10577,2016-06-04,85.1,3.1073,4.8962,1.494075,0.522500,7.0,-11.282,-123.436,-3.230,18.086,23.614,-28.618,84.254,1989.0,6.010,54.4919,57.0,2.0,NaN
10578,2016-06-04,87.4,3.0859,4.9001,1.533042,0.518200,7.0,-13.162,-126.618,-2.837,19.583,23.605,-28.466,87.365,1985.0,6.518,53.9842,57.0,1.0,NaN


In [1141]:
# replace limit on display rows
pd.set_option('display.max_rows', 60)

For whatever reason, Statcast failed to classify the majority of pitches Sale threw that day, beginning with his third pitch of the third inning. (The pitch that I manually labeled a bit earlier at index 10604 also happens to fall into this mix.)

In [1143]:
pitch_name_nan = sale[sale['game_date']=='2016-06-04'].pitch_name.isnull().sum()
total_pitches = len(sale[sale['game_date']=='2016-06-04'])

print("{}% of that day's pitches are unlabeled.".format(round(100 *
                                                             (pitch_name_nan /
                                                              total_pitches),
                                                            2)))

59.46% of that day's pitches are unlabeled.


One approach would be to drop `2016-06-04` altogether—in addition to the three rows from September 7, 2015 where `pitch_name` values are missing.

Instead, I'll attempt to preserve these 69 rows and assign `pitch_name` values by using K-Nearest Neighbors (KNN)..., predict what the actual `pitch_name` is for each observation where `pitch_name` is missing.

The first step here would be to gather together only those columns that would be useful in predicting `pitch_name`, and then separate data where `pitch_name` values are missing from data where `pitch_name` values are not missing.

In [804]:
# STOP
# tighten up the above markdown cell with complete explanation

In [1144]:
pitch_metrics = sale[['release_speed', 'release_pos_x', 'release_pos_z',
                      'pfx_x', 'pfx_z', 'vx0', 'vy0', 'vz0', 'ax', 'ay',
                      'az', 'effective_speed', 'release_spin_rate',
                      'release_extension', 'release_pos_y',
                      'pitch_name']]

In [1145]:
# make subset of data where 'pitch_name' values are not missing
pitch_metrics_pitchname_notnull = pitch_metrics[pitch_metrics['pitch_name'].notnull()]

# make subset of data where 'pitch_name' values are missing
pitch_metrics_pitchname_isnull = pitch_metrics[pitch_metrics['pitch_name'].isnull()]

Now, working with `pitch_metrics_pitchname_notnull`, I'll separate my target (`pitch_name`) from the rest of the data, and then divide each of these into training and testing sets.

In [1146]:
# create data (X)
pitch_metrics_pitchname_notnull_data = pitch_metrics_pitchname_notnull.drop(columns='pitch_name')

# create labels (y)
pitch_metrics_pitchname_notnull_labels = pitch_metrics_pitchname_notnull['pitch_name']

In [1147]:
# split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(pitch_metrics_pitchname_notnull_data,
                                                    pitch_metrics_pitchname_notnull_labels,
                                                    test_size=0.20,
                                                    random_state=77)

Since KNN is a distance-based classifier, if data is in different scales, then larger scaled features have a larger impact on the distance between points.

The remedy for this is to normalize the data, which is done after splitting it into training and test sets so as to avoid "data leakage" (leaking information from the test set to the training set). Normalization (sometimes called Standardization or Scaling) means making sure that all of the data is represented at the same scale. The most common way to do this is to convert all numerical values to z-scores.

To scale my data, I'll use StandardScaler found in the sklearn.preprocessing module.

In [1148]:
# Instantiate StandardScaler
scaler = StandardScaler()

# Transform the training and test sets
scaled_data_train = scaler.fit_transform(X_train)
scaled_data_test = scaler.transform(X_test)

# Convert into a DataFrame
scaled_df_train = pd.DataFrame(scaled_data_train,
                               columns=X_train.columns)
scaled_df_train.head()

,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,vx0,vy0,vz0,ax,ay,az,effective_speed,release_spin_rate,release_extension,release_pos_y
0,-0.482414,-1.475548,-1.250714,0.731752,-0.315307,0.686512,0.460625,1.361331,0.523018,0.597112,-0.601276,-0.506870,-1.605005,0.351917,-0.354282
1,-1.296947,-0.740928,0.285780,-1.676700,-1.565279,1.010176,1.292352,1.138932,-1.618388,-1.051878,-1.373777,-1.252199,0.480163,-0.410396,0.412573
2,-1.180585,0.483440,-0.313756,-1.212836,-1.814741,0.901289,1.186499,0.219925,-1.256930,-1.019924,-1.510998,-1.210374,1.544318,-1.283937,1.283863
3,-1.791485,0.964532,-0.542587,-1.793902,-1.487981,1.880788,1.790672,1.683303,-1.661320,-1.241821,-1.301056,-1.801646,-0.066295,-1.289363,1.294171
4,-0.496959,1.172620,-0.611236,0.556105,-0.720590,-0.993841,0.527965,1.050378,0.332523,-1.174983,-0.734373,-0.456679,-2.069974,-0.301882,0.303526


Now that I've preprocessed the data, it's time to train a KNN classifier and validate its accuracy. With each iteration, the following function will:

- Search for the best value of K,
- fit a classifier to the training data, and...
- generate predictions for X_test (scaled_data_test).

Once predictions are generated using the most optimal K-value, the function will then calculate and print evaluation metrics.

In [1149]:
# define function

def find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=25):
    best_k = 0
    best_score = 0.0
    for k in range(min_k, max_k+1, 2):
        # Instantiate KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors=k)
        # Fit the classifier
        knn.fit(X_train, y_train)
        # Predict on the test set
        preds = knn.predict(X_test)
        accuracy = accuracy_score(y_test, preds)
        precision = precision_score(y_test, preds, average='macro')
        recall = recall_score(y_test, preds, average='macro')
        f1 = f1_score(y_test, preds, average='macro')
        if f1 > best_score:
            best_k = k
            best_score = f1
    
    print("Best Value for k: {}".format(best_k))
    print("Accuracy: {}".format(accuracy))
    print("Precision: {}".format(precision))
    print("Recall: {}".format(recall))
    print("F1-Score: {}".format(best_score))

In [1150]:
find_best_k(scaled_data_train, y_train, scaled_data_test, y_test)

Best Value for k: 11
Accuracy: 0.9178263750828363
Precision: 0.9169031926103746
Recall: 0.9240708226132955
F1-Score: 0.9229966932588936


In [ ]:
# STOP
# make a confusion matrix?

In [ ]:
# Now fill in the nulls by generating preds
# STOP make this a markdown and explain thoroughly what I'm going to do

In [1151]:
# set data apart from missing labels
pitch_metrics_pitchname_isnull_data = pitch_metrics_pitchname_isnull.drop(columns='pitch_name')

# make array with the null labels
pitch_metrics_pitchname_isnull_labels = pitch_metrics_pitchname_isnull['pitch_name']

In [1152]:
# Instantiate StandardScaler
scaler = StandardScaler()

# Transform the data
scaled_data = scaler.fit_transform(pitch_metrics_pitchname_isnull_data)

# Convert into a DataFrame
scaled_df_data = pd.DataFrame(scaled_data,
                              columns=pitch_metrics_pitchname_isnull_data.columns,
                              index=pitch_metrics_pitchname_isnull_data.index)
scaled_df_data.head()

,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,vx0,vy0,vz0,ax,ay,az,effective_speed,release_spin_rate,release_extension,release_pos_y
10569,1.013528,0.198816,0.751191,0.102166,1.069153,-0.170174,-1.026486,-0.269849,0.223162,0.733816,1.166407,0.967798,1.014050,0.043813,-0.042308
10570,-1.305561,-0.720141,0.541596,-2.459493,-2.676180,0.195773,1.323316,1.190864,-2.550003,-0.094600,-2.388619,-1.495215,0.681614,-2.217606,2.216087
10571,1.065063,-0.598431,2.376389,0.369707,1.044533,-0.556808,-1.069892,0.108890,0.530142,2.308022,1.113899,0.919443,1.431005,-0.050553,0.051329
10572,0.996350,-1.710803,0.324238,0.217513,0.964517,0.103038,-1.001682,0.244969,0.342526,1.555449,1.023886,0.984625,1.245066,1.081841,-1.082416
10573,0.515353,1.086638,2.158476,0.320043,-0.020286,0.795697,-0.551636,0.020866,0.351248,1.467891,-0.091397,0.371853,0.011106,-1.017807,1.017673


In [1153]:
# Generate predictions to replace the null values under 'pitch_name'

# Instantiate KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=11)
# Fit the classifier
knn.fit(scaled_data_train, y_train)
# Predict on the data
data_preds = knn.predict(scaled_df_data)

data_preds

array(['4-Seam Fastball', 'Slider', '2-Seam Fastball', '2-Seam Fastball',
       '2-Seam Fastball', '4-Seam Fastball', '4-Seam Fastball',
       'Changeup', 'Changeup', 'Changeup', '2-Seam Fastball', 'Changeup',
       '4-Seam Fastball', '2-Seam Fastball', '2-Seam Fastball',
       '4-Seam Fastball', 'Slider', '2-Seam Fastball', 'Changeup',
       '2-Seam Fastball', '4-Seam Fastball', '2-Seam Fastball', 'Slider',
       'Changeup', '2-Seam Fastball', 'Changeup', 'Slider',
       '2-Seam Fastball', '2-Seam Fastball', 'Changeup',
       '2-Seam Fastball', '2-Seam Fastball', '2-Seam Fastball',
       '2-Seam Fastball', '2-Seam Fastball', '2-Seam Fastball',
       'Changeup', 'Slider', 'Slider', 'Slider', '2-Seam Fastball',
       '2-Seam Fastball', 'Changeup', '4-Seam Fastball',
       '4-Seam Fastball', '2-Seam Fastball', '2-Seam Fastball',
       'Changeup', '2-Seam Fastball', '4-Seam Fastball', 'Changeup',
       'Changeup', '2-Seam Fastball', 'Slider', 'Slider', 'Changeup',
       'Sl

In [1154]:
# Convert new predictions into DataFrame

predictions = pd.DataFrame(data_preds,
                           columns=['pitch_name'],
                           # Set index to match original `Sale` DF
                           # where 'pitch_name' values are missing
                           index=pitch_metrics_pitchname_isnull_labels.index)

predictions.head()

,pitch_name
10569,4-Seam Fastball
10570,Slider
10571,2-Seam Fastball
10572,2-Seam Fastball
10573,2-Seam Fastball


At this point, I'm finally ready to replace the null values in the `pitch_name` of the original `sale` DataFrame with the new predicted values that I've generated.

In [1155]:
sale['pitch_name'].fillna(value=predictions['pitch_name'], inplace=True)

In [1156]:
# check to see that there are no more null values

sale.isnull().sum()

game_date            0
release_speed        0
release_pos_x        0
release_pos_z        0
pfx_x                0
pfx_z                0
inning               0
vx0                  0
vy0                  0
vz0                  0
ax                   0
ay                   0
az                   0
effective_speed      0
release_spin_rate    0
release_extension    0
release_pos_y        0
at_bat_number        0
pitch_number         0
pitch_name           0
dtype: int64

As an extra assurance, I'd like to see that June 4, 2016 looks as it should, with all of its `pitch_name` values in place.

In [1157]:
sale[sale['game_date']=='2016-06-04']

,game_date,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,inning,vx0,vy0,vz0,ax,ay,az,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name
10569,2016-06-04,95.3,3.0566,5.3567,1.090492,1.166067,7.0,-9.472,-138.456,-2.470,17.181,28.400,-18.895,94.613,2328.0,6.003,54.4991,58.0,7.0,4-Seam Fastball
10570,2016-06-04,81.8,2.9592,5.3189,-1.134783,-0.578300,7.0,-8.446,-118.751,0.600,-10.164,25.713,-39.748,79.536,2269.0,5.332,55.1696,58.0,6.0,Slider
10571,2016-06-04,95.6,2.9721,5.6498,1.322900,1.154600,7.0,-10.556,-138.820,-1.674,20.208,33.506,-19.203,94.317,2402.0,5.975,54.5269,58.0,5.0,2-Seam Fastball
10572,2016-06-04,95.2,2.8542,5.2797,1.190692,1.117333,7.0,-8.706,-138.248,-1.388,18.358,31.065,-19.731,94.716,2369.0,6.311,54.1903,58.0,4.0,2-Seam Fastball
10573,2016-06-04,92.4,3.1507,5.6105,1.279758,0.658667,7.0,-6.764,-134.474,-1.859,18.444,30.781,-26.273,90.965,2150.0,5.688,54.8138,58.0,3.0,2-Seam Fastball
10574,2016-06-04,94.4,2.9587,5.4278,1.448150,1.075767,7.0,-13.303,-136.716,-5.533,21.697,27.000,-20.414,93.408,2309.0,5.907,54.5952,58.0,2.0,4-Seam Fastball
10575,2016-06-04,94.8,2.9834,5.2266,1.136417,1.279300,7.0,-12.584,-137.469,-2.881,17.733,25.191,-17.426,94.992,2322.0,6.421,54.0804,58.0,1.0,4-Seam Fastball
10576,2016-06-04,88.9,3.1527,5.2244,1.040392,0.697367,7.0,-9.755,-129.063,-2.178,14.347,24.061,-26.190,88.438,1883.0,6.071,54.4309,57.0,3.0,Changeup
10577,2016-06-04,85.1,3.1073,4.8962,1.494075,0.522500,7.0,-11.282,-123.436,-3.230,18.086,23.614,-28.618,84.254,1989.0,6.010,54.4919,57.0,2.0,Changeup
10578,2016-06-04,87.4,3.0859,4.9001,1.533042,0.518200,7.0,-13.162,-126.618,-2.837,19.583,23.605,-28.466,87.365,1985.0,6.518,53.9842,57.0,1.0,Changeup


Very good! And I've only lost 25 rows along the way.

In [1160]:
sale.head()

,game_date,release_speed,release_pos_x,release_pos_z,pfx_x,pfx_z,inning,vx0,vy0,vz0,ax,ay,az,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name
0,2019-08-13,89.8,3.1853,4.9191,1.4110,-0.0924,7.0,-10.9405,-130.2111,-2.3558,18.0909,26.2440,-32.9637,89.118,1983.0,5.973,54.5262,64.0,2.0,Changeup
1,2019-08-13,81.3,3.2220,5.2825,-1.0911,-0.3883,7.0,-3.1605,-118.1095,2.4188,-9.2399,23.3693,-36.4355,79.805,2624.0,5.435,55.0647,64.0,1.0,Slider
2,2019-08-13,83.1,3.0410,5.2492,-1.1546,-0.4757,7.0,-5.0542,-120.9725,0.6173,-10.2351,23.8790,-37.1534,82.835,2636.0,6.117,54.3824,63.0,5.0,Slider
3,2019-08-13,96.6,3.0312,5.5203,0.9229,0.9892,7.0,-10.2223,-140.3095,-2.9091,14.5585,31.2505,-18.5637,96.925,2422.0,6.540,53.9592,63.0,4.0,4-Seam Fastball
4,2019-08-13,95.7,3.1314,5.2087,1.1255,0.9681,7.0,-12.3400,-138.8856,-2.6033,17.4733,31.9428,-19.1956,95.798,2525.0,6.554,53.9453,63.0,3.0,4-Seam Fastball


In [ ]:
# STOP
# Should I remove outliers? 

In [ ]:
# STOP should i still make new feature for pitch number if I still have
# index as unique identifier?

In [ ]:
# save copy of data
# sale_clean = sale.copy()

## 3. Explore
#### Understand the data. Find significant patterns and trends using statistical methods.

In [ ]:
sale.describe()

In [ ]:
# start wednesday by addressing my STOP comments, filling out thesis some more

In [ ]:
# wafflepy

In [ ]:
# make `at_bat_number` seamless, fill gaps from the side of innings that
# sale didn't pitch

In [ ]:
# make different dfs for each pitch